In [1]:
import os
import time
import sys

import requests

In [2]:
POP20_CC = ('CN IN US ID BR PK NG BD RU JP'
           ' MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'http://flupy.org/data/flags'

DEST_DIR = 'downloads/'

In [4]:
def saveFlag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    if not os.path.exists(DEST_DIR):
        os.makedirs(DEST_DIR)
    with open(path, 'wb') as fp:
        fp.write(img)
        
def getFlag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc = cc.lower())
    resp = requests.get(url)
    return resp.content

def show(text):
    print(text, end = ' ')
    sys.stdout.flush()
    
def downloadMany(ccList):
    for cc in sorted(ccList):
        image = getFlag(cc)
        show(cc)
        saveFlag(image, cc.lower() + '.gif')
        
    return len(ccList)

def main(downloadMany):
    t0 = time.time()
    count = downloadMany(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloads in {:.2f}s'
    print(msg.format(count, elapsed))
    
main(downloadMany)

BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 
20 flags downloads in 17.35s


In [7]:
## 多线程版本
from concurrent import futures

MAX_WOKERS = 20

def downloadOne(cc):
    image = getFlag(cc)
    show(cc)
    saveFlag(image, cc.lower() + '.gif')
    return cc

def downloadManyFutures(ccList):
    wokers = min(MAX_WOKERS, len(ccList))
    with futures.ThreadPoolExecutor(wokers) as executor:
        res = executor.map(downloadOne, sorted(ccList))
    return len(list(res))

main(downloadManyFutures)

DE FR IN TR EG VN NG ID RU JP PK ET MX PH US IR BD BR CN CD 
20 flags downloads in 2.77s


In [16]:
## 一窥future的面貌
def downloadManyFutures2(ccList):
    ccList = ccList[:5]
    with futures.ThreadPoolExecutor(3) as executor:
        toDo = []
        for cc in sorted(ccList):
            future = executor.submit(downloadOne, cc)
            toDo.append(future)
            msg = 'Scheduled for {}: {}'
            print(msg.format(cc, future))
            
        results = []
        for future in futures.as_completed(toDo):
            res = future.result()
            msg = '{} result: {!r}'
            print(msg.format(future, res))
            results.append(res)
        return len(results)
    
main(downloadManyFutures2)

Scheduled for BR: <Future at 0x7fa900402d30 state=running>
Scheduled for CN: <Future at 0x7fa9004029e8 state=running>
Scheduled for ID: <Future at 0x7fa900caa048 state=running>
Scheduled for IN: <Future at 0x7fa900caada0 state=pending>
Scheduled for US: <Future at 0x7fa900caac50 state=pending>
BR <Future at 0x7fa900402d30 state=finished returned str> result: 'BR'ID 
CN <Future at 0x7fa900caa048 state=finished returned str> result: 'ID'
<Future at 0x7fa9004029e8 state=finished returned str> result: 'CN'
IN <Future at 0x7fa900caada0 state=finished returned str> result: 'IN'
US <Future at 0x7fa900caac50 state=finished returned str> result: 'US'

5 flags downloads in 0.99s
